<a href="https://colab.research.google.com/github/SMTorg/smt/blob/master/tutorial/SMT_DesignSpace_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div class="jumbotron text-left"><b>
    
This tutorial describes how to use de DesignSpace within the SMT toolbox. 
<div>
    
    July 2024 - `SMT version 2.6.3`
  
     Paul Saves, Jasper Bussemaker (DLR), and Nathalie BARTOLI (ONERA/DTIS/M2CI)

<div class="alert alert-info fade in" id="d110">
<p>Some updates</p>
<ol> -  Manipulation of mixed DOE (continuous, integer,  categorical and hierarchical variables) </ol>
</div>

<p class="alert alert-success" style="padding:1em">
To use SMT models, please follow this link : https://github.com/SMTorg/SMT/blob/master/README.md. The documentation is available here: http://smt.readthedocs.io/en/latest/
</p>

The reference paper is available 
here https://www.sciencedirect.com/science/article/pii/S0965997818309360?via%3Dihub 

or as a preprint: http://mdolab.engin.umich.edu/content/python-surrogate-modeling-framework-derivatives

For mixed integer with continuous relaxation, the reference paper is available here https://www.sciencedirect.com/science/article/pii/S0925231219315619

In [1]:
!pip install configspace==0.6.1
#!pip install adsg_core --pre
#!pip install adsg-core[nb]

# to install smt
!pip install smt

!pip install git+https://github.com/jbussemaker/adsg-core.git@dev

# to have the latest version
!pip install smt --pre

  Cloning https://github.com/jbussemaker/adsg-core.git (to revision dev) to /tmp/pip-req-build-r6vxrr33
  Running command git clone --filter=blob:none --quiet https://github.com/jbussemaker/adsg-core.git /tmp/pip-req-build-r6vxrr33
  Running command git checkout -b dev --track origin/dev
  Switched to a new branch 'dev'
  Branch 'dev' set up to track remote branch 'dev' from 'origin'.
  Resolved https://github.com/jbussemaker/adsg-core.git to commit 01cb8af781f47701317e7f5ac5f4d82ef300d38f
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


<div class="alert alert-warning" >
If you use hierarchical variables and the size of your doe greater than 30 points, you may leverage the `numba` JIT compiler to speed up the computation
To do so:
    
 - install numba library
    
     `pip install numba`
    
    
 - and define the environment variable `USE_NUMBA_JIT = 1` (unset or 0 if you do not want to use numba) 
    
     - Linux: export USE_NUMBA_JIT = 1
    
     - Windows: set USE_NUMBA_JIT = 1

</div>

In [2]:
%matplotlib inline


from smt.utils.design_space import (
    ArchDesignSpaceGraph,
    DesignSpace,
    FloatVariable,
    IntegerVariable,
    OrdinalVariable,
    CategoricalVariable,
)

import plotly.io as pio

# to ignore warning messages
import warnings

warnings.filterwarnings("ignore")

pio.renderers.default = "notebook"

# Manipulate DOE with mixed, categorical & hierarchical variables

4 variables 
 - 1 categorical variable with 2 labels ['A', 'B'] # x0 categorical: A or B; order is not relevant
 - 1 ordinal variable with 3 levels ['C', 'D', 'E']),  # x1 ordinal: C, D or E; order is relevant
 - 1 integer variable [0,2]: 3 possibilities: 0, 1, 2
 - 1 continuous variable $\in [0, 1]$
 
 
 **Posssibility to have hierarchical variable: x1 exists only if x0 = 'A'**

In [3]:
# Instantiate the design space with all its design variables:

ds = DesignSpace(
    [
        CategoricalVariable(
            ["A", "B"]
        ),  # x0 categorical: A or B; order is not relevant
        OrdinalVariable(["C", "D", "E"]),  # x1 ordinal: C, D or E; order is relevant
        IntegerVariable(0, 2),  # x2 integer between 0 and 2 (inclusive): 0, 1, 2
        FloatVariable(0, 1),  # c3 continuous between 0 and 1
    ]
)

print("Number of design variables", len(ds.design_variables))
# You can define decreed variables (conditional activation):
ds.declare_decreed_var(
    decreed_var=1, meta_var=0, meta_value="A"
)  # Activate x1 if x0 == A

Number of design variables 4


In [4]:
## To give some examples
# It is also possible to randomly sample design vectors conforming to the constraints:
n = 5
x_sampled, is_acting_sampled = ds.sample_valid_x(5)

print("Data encoded: \n", x_sampled)
print("Data in initial space: \n", ds.decode_values(x_sampled))

Data encoded: 
 [[ 1.          0.         -0.          0.94502275]
 [ 1.          0.         -0.          0.65314755]
 [ 1.          0.          1.          0.94803344]
 [ 1.          0.          2.          0.74665591]
 [ 0.          2.          2.          0.62429367]]
Data in initial space: 
 [['B', 'C', -0.0, 0.9450227460759346], ['B', 'C', -0.0, 0.6531475548727604], ['B', 'C', 1.0, 0.9480334401750217], ['B', 'C', 2.0, 0.7466559097987561], ['A', 'E', 2.0, 0.6242936690099832]]


In [5]:
# After defining everything correctly, you can then use the design space object
# to correct design vectors and get information about which design variables are acting:
x_corr, is_acting = ds.correct_get_acting(x_sampled)
print("Which variables are active \n", is_acting)

Which variables are active 
 [[ True False  True  True]
 [ True False  True  True]
 [ True False  True  True]
 [ True False  True  True]
 [ True  True  True  True]]


In [6]:
# If needed, it is possible to get the legacy design space definition format:
xlimits = ds.get_x_limits()
cont_bounds = ds.get_num_bounds()
unfolded_cont_bounds = ds.get_unfolded_num_bounds()
print("Limits of each variable \n", xlimits)
print("Continuous bounds with the encoding done (4 variables now) \n", cont_bounds)
print(
    "Continuous bounds with the unfolded encoding done (5 variables now)\n",
    unfolded_cont_bounds,
)

Limits of each variable 
 [['A', 'B'], ['0', '1', '2'], (0, 2), (0, 1)]
Continuous bounds with the encoding done (4 variables now) 
 [[0 1]
 [0 2]
 [0 2]
 [0 1]]
Continuous bounds with the unfolded encoding done (5 variables now)
 [[0. 1.]
 [0. 1.]
 [0. 2.]
 [0. 2.]
 [0. 1.]]


# Manipulate DOE with continuous variables

In [7]:
# You can also instantiate a purely-continuous design space from bounds directly:
continuous_design_space = DesignSpace([(0, 1), (0, 2), (0.5, 5.5)])
print(
    "Number of design variables =",
    continuous_design_space.n_dv,
    " or ",
    len(continuous_design_space.design_variables),
)

Number of design variables = 3  or  3


In [8]:
x_sampled_cont, is_acting_sampled_cont = continuous_design_space.sample_valid_x(5)

In [9]:
print("Data encoded: \n", x_sampled_cont)
print("Is_acting: \n", is_acting_sampled_cont)

Data encoded: 
 [[0.32273725 1.13803607 0.6039725 ]
 [0.1895724  1.63284148 3.92965975]
 [0.01270165 0.76386364 5.27401859]
 [0.90201543 1.23767912 1.84026453]
 [0.18923128 0.79938574 3.65151063]]
Is_acting: 
 [[ True  True  True]
 [ True  True  True]
 [ True  True  True]
 [ True  True  True]
 [ True  True  True]]


# Moving towards Architecture Design Space Graph (ADSG)

In [10]:
from adsg_core import BasicADSG, NamedNode, DesignVariableNode

# Create the ADSG
adsg = BasicADSG()

ndv = 13


# Create nodes
n = [NamedNode(f"N{i}") for i in range(ndv)]
n = [
    NamedNode("MLP"),
    NamedNode("Learning_rate"),
    NamedNode("Activation_function"),
    NamedNode("Optimizer"),
    NamedNode("Decay"),
    NamedNode("Power_update"),
    NamedNode("Average_start"),
    NamedNode("Running_Average_1"),
    NamedNode("Running_Average_2"),
    NamedNode("Numerical_Stability"),
    NamedNode("Nb_layers"),
    NamedNode("Layer_1"),
    NamedNode("Layer_2"),
    NamedNode("Layer_3"),  # NamedNode("Dropout"),
    NamedNode("ASGD"),
    NamedNode("Adam"),
    NamedNode("20...40"),
    NamedNode("40"),
    NamedNode("45"),
    NamedNode("20...40"),
    NamedNode("40"),
    NamedNode("45"),
    NamedNode("20...40"),
    NamedNode("40"),
    NamedNode("45"),
]
adsg.add_node(n[1])
adsg.add_node(n[2])
# adsg.add_node(n[3])

# Add some edges
adsg.add_edges(
    [
        # (n[0], n[1]),
        # (n[0], n[2]),
        # (n[0], n[3]),
        (n[3], n[10]),
        (n[14], n[4]),
        (n[14], n[5]),
        (n[14], n[6]),
        (n[15], n[7]),
        (n[15], n[8]),
        (n[15], n[9]),
    ]
)

choiceo = adsg.add_selection_choice("Optimizer_Choice", n[3], [n[14], n[15]])


choicenl = adsg.add_selection_choice("#layers", n[10], [n[11], n[12], n[13]])
# adsg.add_edges([ (n[12],n[11]), (n[13],n[12]) ])
a = []
for i in range(3):
    a.append(NamedNode(str(25 + 5 * i)))
b = a.copy()
b.append(n[17])
b.append(n[18])
choicel1 = adsg.add_selection_choice("#neurons_1", n[11], b)
adsg.add_edges([(n[12], choicel1), (n[13], choicel1)])


a = []
for i in range(3):
    a.append(NamedNode(str(25 + 5 * i)))
b = a.copy()
b.append(n[20])
b.append(n[21])
choicel1 = adsg.add_selection_choice("#neurons_2", n[12], b)
adsg.add_edges([(n[13], choicel1)])

a = []
for i in range(3):
    a.append(NamedNode(str(25 + 5 * i)))
b = a.copy()
b.append(n[23])
b.append(n[24])
choicel1 = adsg.add_selection_choice("#neurons_3", n[13], b)

# adsg.add_edges([ (n[18],n[17]), (n[17],n[16]) ])
# adsg.add_edges([ (n[21],n[20]), (n[20],n[19]) ])
# adsg.add_edges([ (n[24],n[23]), (n[23],n[22]) ])

adsg.add_incompatibility_constraint([n[15], n[13]])
adsg.add_incompatibility_constraint([n[14], n[17]])
adsg.add_incompatibility_constraint([n[14], n[18]])
adsg.add_incompatibility_constraint([n[14], n[20]])
adsg.add_incompatibility_constraint([n[14], n[21]])
adsg.add_incompatibility_constraint([n[14], n[23]])
adsg.add_incompatibility_constraint([n[14], n[24]])
start_nodes = set()
start_nodes.add(n[3])
start_nodes.add(n[2])
start_nodes.add(n[1])
# start_nodes.add(n[0])


adsg.add_edges(
    [
        (n[1], DesignVariableNode("x0", bounds=(0, 1))),
        (n[4], DesignVariableNode("x1", bounds=(0, 1))),
        (n[5], DesignVariableNode("x2", bounds=(0, 1))),
        (n[6], DesignVariableNode("x3", bounds=(0, 1))),
        (n[7], DesignVariableNode("x4", bounds=(0, 1))),
        (n[8], DesignVariableNode("x5", bounds=(0, 1))),
        (n[9], DesignVariableNode("x6", bounds=(0, 1))),
        # (n[11], DesignVariableNode('x7', options=("0", "1"))),
    ]
)

choiceo = adsg.add_selection_choice(
    "Activation_Choice",
    n[2],
    [NamedNode("ReLU"), NamedNode("Sigmoid"), NamedNode("Tanh")],
)

adsg = adsg.set_start_nodes(start_nodes)
adsg.render()

In [11]:
from adsg_core import GraphProcessor

gp = GraphProcessor(adsg)

print("Design variables:", gp.des_vars[0:5])
print(str(gp.des_vars[5:])[1:])
print("Objectives:", gp.objectives)
print("Constraints:", gp.constraints)

# Display some details about the encoders used for
# formulating the optimization problem
gp.get_statistics()

Design variables: [DV: #layers [3 opts], DV: Activation_Choice [3 opts], DV: Optimizer_Choice [2 opts], DV: #neurons_1 [5 opts], DV: #neurons_2 [5 opts]]
DV: #neurons_3 [5 opts], DV: x0 [0.00..1.00], DV: x1 [0.00..1.00], DV: x2 [0.00..1.00], DV: x3 [0.00..1.00], DV: x4 [0.00..1.00], DV: x5 [0.00..1.00], DV: x6 [0.00..1.00]]
Objectives: []
Constraints: []


,n_valid,n_declared,n_discrete,n_dim_cont,n_dim_cont_mean,n_exist,imp_ratio,imp_ratio_comb,imp_ratio_cont,inf_idx,dist_corr,encoder
type,,,,,,,,,,,,
option-decisions,207,1350,6,0,0.0,1,6.521739,6.521739,1.00,0.531986,0.0,complete
additional-dvs,207,0,0,7,4.0,207,1.750000,1.000000,1.75,1.000000,0.0,
total-design-space,207,2250,6,7,4.0,1,19.021739,10.869565,1.75,0.493305,0.0,complete
total-design-problem,207,2250,6,7,4.0,1,19.021739,10.869565,1.75,0.493305,0.0,complete


In [12]:
design_space = ArchDesignSpaceGraph(adsg=adsg)
design_space._sample_valid_x(1, return_render=True)[2][0].render()